In [1]:
from data_processing import *
from model_training import *
import glob
import math
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import warnings
from os import path
warnings.filterwarnings("ignore")

In [2]:
columns = pd.read_csv('data/columns.txt', header=None)
columns_list = columns.transpose().values.tolist()[0]

sensors = ['lJPos', 'rJPos', 'lJVel',
           'rJVel', 'gyroX', 'gyroY', 'gyroZ', 'accX',
           'accY', 'accZ']

mode_pool = ['L0R0']
trial_pool = [1, 2, 3, 4, 5]
data_all = []

data_dir = 'data/'
for mode in mode_pool:
    for trial in trial_pool:
        file_path = data_dir+'log_'+str(mode)+'_'+str(trial)+'.txt'
        if path.exists(file_path) == 1:
            for read_path in glob.glob(file_path):
                data = pd.read_csv(read_path, sep=" ", header=None)
                data = data.iloc[:,0:15]
                data.columns = columns_list
                data = data[sensors]
                data_all.append(data)
data_list = label_data(data_all)

In [12]:
## CNN Model
hyperparam_space = {
    'window_size': [30],
    'model': 'cnn',
    'cnn': {
      'kernel_size': [20],
      'activation': ['relu']
    },
    'dense': {
        'num_layers': [2],
        'num_nodes': [5],
        'activation': ['tanh']
    },
    'optimizer': {
        'loss': ['mean_absolute_error'],
        'optimizer': ['adam']
    },
    'training': {
        'epochs': [2],
        'batch_size': [128]
    }
}
hyperparameter_configs = get_model_configs(hyperparam_space)

In [13]:
trial_results, average_results = train_models(hyperparam_space['model'], hyperparameter_configs, data_list)

(20942, 30, 10)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 30, 10)            21        
_________________________________________________________________
conv1d (Conv1D)              (None, 11, 10)            2010      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1, 10)             1110      
_________________________________________________________________
activation (Activation)      (None, 1, 10)             0         
_________________________________________________________________
flatten (Flatten)            (None, 10)                0         
_________________________________________________________________
dense (Dense)                (None, 4)                 44        
Total params: 3,185
Trainable params: 3,164
Non-trainable params: 21
_____________________________________

In [14]:
display(average_results)

,window_size,model_type,cnn_kernel_size,cnn_activation,dense_num_layers,dense_num_nodes,dense_activation,optim_loss,optim_optimizer,training_epochs,training_batch_size,left_rmse_mean,right_rmse_mean
0,30,cnn,20,relu,2,5,tanh,mean_absolute_error,adam,2,128,11.595101,9.249293


In [ ]:
## LSTM Model
# hyperparam_space = {
#     'window_size': [20],
#     'model': 'lstm',
#     'lstm': {
#       'units': [30],
#       'activation': ['relu']
#     },
#     'dense': {
#         'activation': ['tanh']
#     },
#     'optimizer': {
#         'loss': ['mean_absolute_error'],
#         'optimizer': ['adam']
#     },
#     'training': {
#         'epochs': [10],
#         'batch_size': [128]
#     }
# }

## CNN Model
# hyperparam_space = {
#     'window_size': [20],
#     'model': 'cnn',
#     'cnn': {
#       'kernel_size': [10],
#       'activation': ['relu']
#     },
#     'dense': {
#         'activation': ['tanh']
#     },
#     'optimizer': {
#         'loss': ['mean_absolute_error'],
#         'optimizer': ['adam']
#     },
#     'training': {
#         'epochs': [10],
#         'batch_size': [128]
#     }
# }

## MLP Model
# hyperparam_space = {
#     'window_size': [20],
#     'model': 'mlp',
#     'dense': {
#         'num_layers': [10, 20],
#         'num_nodes': [5, 10],
#         'activation': ['tanh']
#     },
#     'optimizer': {
#         'loss': ['mean_absolute_error'],
#         'optimizer': ['adam']
#     },
#     'training': {
#         'epochs': [10],
#         'batch_size': [128]
#     }
# }